# Key-Value Store Performance Testing

このノートブックは各Key-Valueストアサービスの性能テストを自動化します。

## 対象サービス
1. **1-coordinator-ring**: コーディネーターリング実装
2. **2-quorum-consistency**: クォーラム一貫性実装
3. **3-sharding-replica**: シャーディング・レプリカ実装
4. **4-distributed-lock**: 分散ロック実装
5. **5-cache-aside**: キャッシュアサイド実装
6. **6-bloom-sstable**: BloomフィルタとSSTable実装
7. **7-rate-limiting**: レートリミティング実装
8. **8-line-streams**: Line風ストリーム実装
9. **9-session-store**: セッションストア実装
10. **10-leaderboard**: リーダーボード実装

## テストパターン
- **軽負荷**: 10 clients, 1000 requests
- **中負荷**: 50 clients, 10000 requests  
- **高負荷**: 100 clients, 50000 requests

In [1]:
import time
import json
import subprocess
import requests
import statistics
import pandas as pd
import numpy as np
from datetime import datetime
import logging
from pathlib import Path
import docker
from contextlib import contextmanager
import requests

# ログ設定
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# パス設定
BASE_DIR = Path("/Users/codefox/workspace/practice_infra_arch/key_value_store")
DATA_DIR = BASE_DIR / "analysis" / "data"
DATA_DIR.mkdir(parents=True, exist_ok=True)

LOG_DIR = BASE_DIR / "analysis" / "logs"
LOG_DIR.mkdir(exist_ok=True)

print("✅ 必要なライブラリがインポートされました")

✅ 必要なライブラリがインポートされました


In [2]:
# --- SERVICES 定義 ---
SERVICES = {
    "1-coordinator-ring": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "write": "/write",
            "read": "/read"
        },
        "test_data": {"key": "test_key", "value": "test_value"}
    },
    "2-quorum-consistency": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "write": "/write",
            "read": "/read"
        },
        "test_data": {"key": "test_key", "value": "test_value"}
    },
    "3-sharding-replica": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "write": "/write",
            "read": "/read"
        },
        "test_data": {"key": "test_key", "value": "test_value"}
    },
    "4-distributed-lock": {
        "port": 8000,
        "endpoints": {
            "health": "/stats",
            "acquire": "/acquire",
            "release": "/release"
        },
        "test_data": {"key": "test_resource", "owner": "test_client"}
    },
    "5-cache-aside": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "get": "/get",
            "set": "/set"
        },
        "test_data": {"entity_type": "user", "entity_id": "1", "data": {"name": "test"}}
    },
    "6-bloom-sstable": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "write": "/write",
            "read": "/read"
        },
        "test_data": {"key": "test_key", "value": "test_value"}
    },
    "7-rate-limiting": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "check": "/check_rate"
        },
        "test_data": {"user_id": "test_user"}
    },
    "8-line-streams": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "produce": "/produce",
            "consume": "/consume"
        },
        "test_data": {"message": "test_message", "consumer": "test_consumer"}
    },
    "9-session-store": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "login": "/login",
            "me": "/me"
        },
        "test_data": {"username": "test_user"}
    },
    "10-leaderboard": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "score": "/score",
            "top": "/top/10"
        },
        "test_data": {"user_id": "test_user", "score": 100}
    }
}

print("SERVICES configured with", len(SERVICES), "services")

SERVICES configured with 10 services


In [3]:
TEST_PATTERNS = {
    "light": {
        "description": "軽負荷",
        "clients": 10,
        "requests": 1000,
        "timeout": 30,
        "ramp_up": 5
    },
    "medium": {
        "description": "中負荷",
        "clients": 50, 
        "requests": 10000,
        "timeout": 60,
        "ramp_up": 10
    },
    "heavy": {
        "description": "高負荷",
        "clients": 100,
        "requests": 50000,
        "timeout": 120,
        "ramp_up": 20
    }
}

In [ ]:
class DockerManager:
    """Dockerコンテナの管理クラス"""
    
    def __init__(self, service_name):
        self.service_name = service_name
        self.service_dir = BASE_DIR / service_name
        self.client = docker.from_env()
        
    def start_service(self):
        """サービスを起動"""
        try:
            cmd = ["docker", "compose", "up", "-d", "--build"]
            result = subprocess.run(
                cmd, cwd=self.service_dir,
                capture_output=True, text=True, timeout=300
            )
            time.sleep(30)
            if result.returncode != 0:
                logger.error(f"❌ {self.service_name} 起動失敗: {result.stderr}")
                self.save_container_logs()
                return False
            return True
        except subprocess.TimeoutExpired:
            logger.error(f"❌ {self.service_name} 起動タイムアウト")
            self.save_container_logs()
            return False
        except Exception as e:
            logger.error(f"❌ {self.service_name} 起動中にエラー: {str(e)}")
            self.save_container_logs()
            return False
    
    def stop_service(self):
        """サービスを停止"""
        try:
            cmd = ["docker", "compose", "down", "-v"]
            result = subprocess.run(
                cmd, cwd=self.service_dir,
                capture_output=True, text=True, timeout=60
            )
            if result.returncode != 0:
                logger.warning(f"⚠️ {self.service_name} 停止時に警告: {result.stderr}")
        except Exception as e:
            logger.error(f"❌ {self.service_name} 停止中にエラー: {str(e)}")
    
    def health_check(self, max_retries=60):
        """ヘルスチェック"""
        service_config = SERVICES[self.service_name]
        health_url = f"http://localhost:{service_config['port']}{service_config['endpoints']['health']}"

        if self.service_name in ['5-cache-aside', '3-sharding-replica']:
            max_retries = 90  # DB依存サービスは長めに待機

        for attempt in range(max_retries):
            try:
                response = requests.get(health_url, timeout=5)
                if response.status_code == 200:
                    return True
            except requests.exceptions.RequestException:
                pass
            time.sleep(2)
        
        logger.error(f"❌ {self.service_name} ヘルスチェック失敗 ({max_retries}回試行)")
        self.save_container_logs()
        return False
    
    def save_container_logs(self):
        """コンテナのエラーログをファイル保存"""
        try:
            containers = self.client.containers.list(
                filters={"label": f"com.docker.compose.project={self.service_name.replace('-', '')}"}
            )
            for container in containers:
                logs = container.logs(tail=200).decode("utf-8", errors="ignore")
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                log_file = LOG_DIR / f"{self.service_name}_{container.name}_{timestamp}.log"
                with open(log_file, "w") as f:
                    f.write(logs)
                logger.error(f"📄 {self.service_name} ({container.name}) のログを保存しました: {log_file}")
        except Exception as e:
            logger.warning(f"⚠️ {self.service_name} ログ保存失敗: {str(e)}")

    def get_container_stats(self):
        """コンテナの統計情報を取得"""
        try:
            containers = self.client.containers.list(
                filters={"label": f"com.docker.compose.project={self.service_name.replace('-', '')}"}
            )
            stats = {}
            for container in containers:
                container_stats = container.stats(stream=False)
                stats[container.name] = {
                    "cpu_percent": self._calculate_cpu_percent(container_stats),
                    "memory_usage": container_stats["memory_stats"]["usage"],
                    "memory_limit": container_stats["memory_stats"]["limit"]
                }
            return stats
        except Exception as e:
            logger.warning(f"⚠️ {self.service_name} 統計情報取得失敗: {str(e)}")
            return {}

    def _calculate_cpu_percent(self, stats):
        """CPU使用率を計算"""
        try:
            cpu_delta = (
                stats["cpu_stats"]["cpu_usage"]["total_usage"]
                - stats["precpu_stats"]["cpu_usage"]["total_usage"]
            )
            system_delta = (
                stats["cpu_stats"]["system_cpu_usage"]
                - stats["precpu_stats"]["system_cpu_usage"]
            )
            num_cpus = len(stats["cpu_stats"]["cpu_usage"]["percpu_usage"])
            if system_delta > 0:
                return (cpu_delta / system_delta) * num_cpus * 100.0
        except (KeyError, ZeroDivisionError):
            pass
        return 0.0



@contextmanager
def service_context(service_name, cooldown=30):
    """サービスのライフサイクル管理"""
    docker_manager = DockerManager(service_name)
    docker_manager.stop_service()
    
    try:
        if not docker_manager.start_service():
            raise RuntimeError(f"Failed to start {service_name}")
        if not docker_manager.health_check():
            raise RuntimeError(f"Health check failed for {service_name}")
        yield docker_manager
    finally:
        docker_manager.stop_service()
        if cooldown > 0:
            time.sleep(cooldown)


def run_service_test(service_name, retries=1):
    """1サービスのテスト（リトライ付き）"""
    for attempt in range(retries + 1):
        try:
            with service_context(service_name):
                return True
        except Exception as e:
            logger.error(f"❌ {service_name} テスト失敗 (試行 {attempt+1}/{retries+1}): {e}")
            if attempt < retries:
                time.sleep(5)
    return False


def test_all_services(retries=1):
    """全サービスを逐次テストし、結果を丸バツ表とJSONで保存"""
    results = {}
    for service_name in SERVICES.keys():
        success = run_service_test(service_name, retries=retries)
        results[service_name] = success

    # 結果を一覧表で出力
    print("\n=== 📝 テスト結果一覧 ===")
    for service, success in results.items():
        mark = "⭕" if success else "❌"
        print(f"{service:20} : {mark}")

    # JSON保存
    result_file = BASE_DIR / "test_results.json"
    with open(result_file, "w") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    print(f"\n💾 結果を保存しました: {result_file}")

    return results


In [5]:
class PerformanceTester:
    """性能テスト実行クラス"""

    def __init__(self, service_name):
        self.service_name = service_name
        self.service_config = SERVICES[service_name]
        self.base_url = f"http://localhost:{self.service_config['port']}"
        self.results = []
        self.session = requests.Session()  # セッション管理用

    def run_single_request(self, endpoint, method="GET", data=None, session=None):
        """単一リクエストの実行と測定"""
        use_session = session if session else (
            self.session if self.service_name == '9-session-store' else None
        )

        url = f"{self.base_url}{endpoint}"
        start_time = time.time()

        try:
            if method == "POST":
                response = (use_session or requests).post(url, json=data, timeout=10)
            else:
                response = (use_session or requests).get(url, params=data, timeout=10)

            response_time = (time.time() - start_time) * 1000  # ミリ秒

            result = {
                "response_time": response_time,
                "status_code": response.status_code,
                "success": 200 <= response.status_code < 300,
                "error": None
            }

            # 9-session-store の場合はセッションIDを保持
            if (self.service_name == '9-session-store' and 
                endpoint == '/login' and 
                response.status_code == 200 and 
                use_session is not None):
                try:
                    response_data = response.json()
                    if 'session_id' in response_data:
                        use_session.cookies.set('session_id', response_data['session_id'])
                except Exception:
                    pass  # JSON パース失敗は無視

            return result

        except Exception as e:
            return {
                "response_time": (time.time() - start_time) * 1000,
                "status_code": 0,
                "success": False,
                "error": str(e)
            }

    def run_load_test(self, pattern_name, docker_manager):
        """負荷テストの実行（逐次処理版）"""
        pattern = TEST_PATTERNS[pattern_name]
        clients = pattern["clients"]
        total_requests = pattern["requests"]
        requests_per_client = total_requests // clients

        print(f"🧪 {self.service_name} - {pattern['description']} 開始")
        print(f"   クライアント数: {clients}, 総リクエスト数: {total_requests}")

        # テスト対象エンドポイント決定
        test_endpoint, test_method, test_data = self._get_test_endpoint()

        # 9-session-store の場合は事前ログイン
        if self.service_name == '9-session-store':
            login_result = self.run_single_request('/login', 'POST', {'username': 'test_user'})
            if not login_result['success']:
                raise RuntimeError(f"Login failed for {self.service_name}")

        initial_stats = docker_manager.get_container_stats()
        start_time = time.time()

        all_results = []
        for client_id in range(clients):
            client_results = self._client_worker(
                client_id, requests_per_client, test_endpoint, test_method, test_data
            )
            all_results.extend(client_results)

        duration = time.time() - start_time
        final_stats = docker_manager.get_container_stats()

        analysis = self._analyze_results(all_results, duration, initial_stats, final_stats)
        analysis.update({
            "service_name": self.service_name,
            "pattern_name": pattern_name,
            "pattern_description": pattern["description"],
            "timestamp": datetime.now().isoformat(),
            "clients": clients,
            "total_requests": len(all_results),
            "duration": duration
        })

        print(f"✅ {self.service_name} - {pattern['description']} 完了")
        print(f"   実行時間: {duration:.2f}秒, QPS: {analysis['qps']:.2f}")

        return analysis

    def _get_test_endpoint(self):
        """サービスごとのテスト対象エンドポイントを決定"""
        endpoints = self.service_config["endpoints"]
        test_data = self.service_config["test_data"]

        if self.service_name == '9-session-store':
            return endpoints["me"], "GET", None
        elif self.service_name == '5-cache-aside':
            entity_type = test_data['entity_type']
            entity_id = test_data['entity_id']
            return f"/set/{entity_type}/{entity_id}", "POST", test_data['data']
        elif "write" in endpoints:
            return endpoints["write"], "POST", test_data
        elif "put" in endpoints:
            return endpoints["put"], "POST", test_data
        elif "publish" in endpoints:
            return endpoints["publish"], "POST", test_data
        elif "produce" in endpoints:
            return endpoints["produce"], "POST", test_data
        elif "acquire" in endpoints:
            return endpoints["acquire"], "POST", test_data
        elif "score" in endpoints:
            return endpoints["score"], "POST", test_data
        else:
            return endpoints["health"], "GET", None

    def _client_worker(self, client_id, requests_count, endpoint, method, data):
        """クライアントごとの逐次リクエスト処理"""
        results = []

        # 9-session-store はクライアントごとにログイン
        if self.service_name == '9-session-store':
            client_session = requests.Session()
            login_data = {'username': f'test_user_{client_id}'}
            login_result = self.run_single_request('/login', 'POST', login_data, client_session)
            if not login_result['success']:
                return []
        else:
            client_session = None

        for i in range(requests_count):
            test_data = self._prepare_dynamic_data(data, client_id, i)
            current_endpoint = endpoint

            # 5-cache-aside は entity_id を動的変更
            if self.service_name == '5-cache-aside':
                etype = self.service_config['test_data']['entity_type']
                eid = f"{self.service_config['test_data']['entity_id']}_{client_id}_{i}"
                current_endpoint = f"/set/{etype}/{eid}"

            result = self.run_single_request(current_endpoint, method, test_data, client_session)
            result.update({"client_id": client_id, "request_id": i})
            results.append(result)

        return results

    def _prepare_dynamic_data(self, data, client_id, i):
        """テストデータを動的に生成"""
        if not data:
            return None

        test_data = data.copy()
        if "key" in test_data:
            test_data["key"] = f"{test_data['key']}_{client_id}_{i}"
        if "resource" in test_data:
            test_data["resource"] = f"{test_data['resource']}_{client_id}_{i}"
        if "client_id" in test_data:
            test_data["client_id"] = f"{test_data['client_id']}_{client_id}"
        if "user_id" in test_data:
            test_data["user_id"] = f"{test_data['user_id']}_{client_id}_{i}"
        if "score" in test_data:
            test_data["score"] = test_data["score"] + i
        return test_data

    def _analyze_results(self, results, duration, initial_stats, final_stats):
        """結果分析"""
        successful = [r for r in results if r["success"]]
        times = [r["response_time"] for r in successful]

        if not times:
            return {
                "total_requests": len(results),
                "successful_requests": 0,
                "error_rate": 100.0,
                "qps": 0.0,
                "avg_response_time": 0.0,
                "median_response_time": 0.0,
                "p95_response_time": 0.0,
                "p99_response_time": 0.0,
                "min_response_time": 0.0,
                "max_response_time": 0.0
            }

        return {
            "total_requests": len(results),
            "successful_requests": len(successful),
            "error_rate": ((len(results) - len(successful)) / len(results)) * 100,
            "qps": len(successful) / duration,
            "avg_response_time": statistics.mean(times),
            "median_response_time": statistics.median(times),
            "p95_response_time": np.percentile(times, 95),
            "p99_response_time": np.percentile(times, 99),
            "min_response_time": min(times),
            "max_response_time": max(times),
            "initial_stats": initial_stats,
            "final_stats": final_stats
        }

print("✅ 性能テストクラス（逐次処理版）が定義されました")


✅ 性能テストクラス（逐次処理版）が定義されました


In [6]:
def run_comprehensive_test(selected_services=None, selected_patterns=None):
    """包括的な性能テストの実行"""
    if selected_services is None:
        selected_services = list(SERVICES.keys())
    if selected_patterns is None:
        selected_patterns = list(TEST_PATTERNS.keys())
    
    all_results = []
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    logger.info(f"🚀 包括的性能テスト開始 - {timestamp}")
    logger.info(f"対象サービス: {selected_services}")
    logger.info(f"テストパターン: {selected_patterns}")
    
    total_tests = len(selected_services) * len(selected_patterns)
    current_test = 0
    
    for service_name in selected_services:
        logger.info(f"\n📊 {service_name} のテスト開始")
        
        try:
            with service_context(service_name) as docker_manager:
                tester = PerformanceTester(service_name)
                
                for pattern_name in selected_patterns:
                    current_test += 1
                    logger.info(f"\n進行状況: {current_test}/{total_tests}")
                    
                    try:
                        result = tester.run_load_test(pattern_name, docker_manager)
                        all_results.append(result)
                        
                        # 中間結果保存
                        save_results([result], f"intermediate_{service_name}_{pattern_name}_{timestamp}")
                        
                        # テスト間の休憩
                        if current_test < total_tests:
                            logger.info("⏸️ 5秒間の休憩...")
                            time.sleep(5)
                            
                    except Exception as e:
                        logger.error(f"❌ {service_name} - {pattern_name} テスト失敗: {str(e)}")
                        # エラー結果も記録
                        error_result = {
                            "service_name": service_name,
                            "pattern_name": pattern_name,
                            "error": str(e),
                            "timestamp": datetime.now().isoformat()
                        }
                        all_results.append(error_result)
                        
        except Exception as e:
            logger.error(f"❌ {service_name} サービス全体のテスト失敗: {str(e)}")
            continue
    
    # 最終結果保存
    save_results(all_results, f"comprehensive_test_{timestamp}")
    
    logger.info(f"\n🎉 全テスト完了! 結果は {DATA_DIR} に保存されました")
    return all_results

def save_results(results, filename_prefix):
    """結果をファイルに保存"""
    # JSON形式で保存
    json_file = DATA_DIR / f"{filename_prefix}.json"
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    
    # CSV形式で保存（数値データのみ）
    csv_file = DATA_DIR / f"{filename_prefix}.csv"
    
    if results and not any('error' in r for r in results):
        # エラーがない場合のみCSV出力
        df = pd.DataFrame(results)
        # 複雑なオブジェクトを除外
        numeric_columns = df.select_dtypes(include=[np.number, 'object']).columns
        simple_columns = [col for col in numeric_columns 
                         if col not in ['initial_stats', 'final_stats']]
        df[simple_columns].to_csv(csv_file, index=False)
    
    logger.info(f"💾 結果保存完了: {json_file}")

print("✅ テスト実行関数が定義されました")

✅ テスト実行関数が定義されました


In [7]:
# ユーザーインターフェース用関数

def show_services():
    """利用可能なサービス一覧を表示"""
    print("📋 利用可能なサービス:")
    for i, (name, config) in enumerate(SERVICES.items(), 1):
        print(f"  {i}. {name} (ポート: {config['port']})")

def show_patterns():
    """利用可能なテストパターン一覧を表示"""
    print("📋 利用可能なテストパターン:")
    for i, (name, pattern) in enumerate(TEST_PATTERNS.items(), 1):
        print(f"  {i}. {name}: {pattern['description']} ({pattern['clients']} clients, {pattern['requests']} requests)")

def quick_test(service_name, pattern_name="light"):
    """単一サービスの簡易テスト"""
    if service_name not in SERVICES:
        print(f"❌ サービス '{service_name}' が見つかりません")
        show_services()
        return None
    
    if pattern_name not in TEST_PATTERNS:
        print(f"❌ テストパターン '{pattern_name}' が見つかりません")
        show_patterns()
        return None
    
    return run_comprehensive_test([service_name], [pattern_name])

def test_all_services(pattern_name="light"):
    """全サービスの指定パターンでのテスト"""
    if pattern_name not in TEST_PATTERNS:
        print(f"❌ テストパターン '{pattern_name}' が見つかりません")
        show_patterns()
        return None
    
    return run_comprehensive_test(None, [pattern_name])

def test_service_all_patterns(service_name):
    """指定サービスの全パターンでのテスト"""
    if service_name not in SERVICES:
        print(f"❌ サービス '{service_name}' が見つかりません")
        show_services()
        return None
    
    return run_comprehensive_test([service_name], None)

print("✅ ユーザーインターフェース関数が定義されました")
print("\n🎯 使用方法:")
print("  - show_services(): サービス一覧表示")
print("  - show_patterns(): テストパターン一覧表示")
print("  - quick_test('service_name', 'pattern_name'): 単一サービステスト")
print("  - test_all_services('pattern_name'): 全サービステスト")
print("  - test_service_all_patterns('service_name'): 指定サービス全パターンテスト")
print("  - run_comprehensive_test(): 完全なカスタムテスト")

✅ ユーザーインターフェース関数が定義されました

🎯 使用方法:
  - show_services(): サービス一覧表示
  - show_patterns(): テストパターン一覧表示
  - quick_test('service_name', 'pattern_name'): 単一サービステスト
  - test_all_services('pattern_name'): 全サービステスト
  - test_service_all_patterns('service_name'): 指定サービス全パターンテスト
  - run_comprehensive_test(): 完全なカスタムテスト


## 🚀 テスト実行例

以下のセルを実行してテストを開始してください。テスト前に必要に応じて設定を確認・変更できます。

In [8]:
# 利用可能なオプションを表示
show_services()
print()
show_patterns()

📋 利用可能なサービス:
  1. 1-coordinator-ring (ポート: 8000)
  2. 2-quorum-consistency (ポート: 8000)
  3. 3-sharding-replica (ポート: 8000)
  4. 4-distributed-lock (ポート: 8000)
  5. 5-cache-aside (ポート: 8000)
  6. 6-bloom-sstable (ポート: 8000)
  7. 7-rate-limiting (ポート: 8000)
  8. 8-line-streams (ポート: 8000)
  9. 9-session-store (ポート: 8000)
  10. 10-leaderboard (ポート: 8000)

📋 利用可能なテストパターン:
  1. light: 軽負荷 (10 clients, 1000 requests)
  2. medium: 中負荷 (50 clients, 10000 requests)
  3. heavy: 高負荷 (100 clients, 50000 requests)


In [ ]:
print("\n✅ テストスクリプトの準備が完了しました。必要に応じて上記の関数を呼び出してテストを実行してください。")
test_all_services(pattern_name="medium")  # 例: 全サービスの中負荷テストを実行

2025-09-11 20:38:42,254 - INFO - 🚀 包括的性能テスト開始 - 20250911_203842
2025-09-11 20:38:42,255 - INFO - 対象サービス: ['1-coordinator-ring', '2-quorum-consistency', '3-sharding-replica', '4-distributed-lock', '5-cache-aside', '6-bloom-sstable', '7-rate-limiting', '8-line-streams', '9-session-store', '10-leaderboard']
2025-09-11 20:38:42,256 - INFO - テストパターン: ['light']
2025-09-11 20:38:42,257 - INFO - 
📊 1-coordinator-ring のテスト開始



✅ テストスクリプトの準備が完了しました。必要に応じて上記の関数を呼び出してテストを実行してください。


2025-09-11 20:38:46,246 - INFO - 
進行状況: 1/10


🧪 1-coordinator-ring - 軽負荷 開始
   クライアント数: 10, 総リクエスト数: 1000


2025-09-11 20:38:49,575 - INFO - 💾 結果保存完了: /Users/codefox/workspace/practice_infra_arch/key_value_store/analysis/data/intermediate_1-coordinator-ring_light_20250911_203842.json
2025-09-11 20:38:49,576 - INFO - ⏸️ 5秒間の休憩...


...

🧪 10-leaderboard - 軽負荷 開始
   クライアント数: 10, 総リクエスト数: 1000


2025-09-11 20:48:19,953 - INFO - 💾 結果保存完了: /Users/codefox/workspace/practice_infra_arch/key_value_store/analysis/data/intermediate_10-leaderboard_light_20250911_203842.json
2025-09-11 20:48:19,953 - INFO - ⏸️ 5秒間の休憩...


✅ 10-leaderboard - 軽負荷 完了
   実行時間: 3.41秒, QPS: 293.48


2025-09-11 20:48:55,370 - INFO - 💾 結果保存完了: /Users/codefox/workspace/practice_infra_arch/key_value_store/analysis/data/comprehensive_test_20250911_203842.json
2025-09-11 20:48:55,371 - INFO - 
🎉 全テスト完了! 結果は /Users/codefox/workspace/practice_infra_arch/key_value_store/analysis/data に保存されました


[{'total_requests': 1000,
  'successful_requests': 1000,
  'error_rate': 0.0,
  'qps': 302.1061263531756,
  'avg_response_time': 3.2771332263946533,
  'median_response_time': 3.1409263610839844,
  'p95_response_time': np.float64(4.7441720962524405),
  'p99_response_time': np.float64(5.8058762550353995),
  'min_response_time': 1.6918182373046875,
  'max_response_time': 8.913993835449219,
  'initial_stats': {},
  'final_stats': {},
  'service_name': '1-coordinator-ring',
  'pattern_name': 'light',
  'pattern_description': '軽負荷',
  'timestamp': '2025-09-11T20:38:49.568975',
  'clients': 10,
  'duration': 3.3100950717926025},
 {'total_requests': 1000,
  'successful_requests': 1000,
  'error_rate': 0.0,
  'qps': 175.59728054107842,
  'avg_response_time': 5.6431193351745605,
  'median_response_time': 5.6324005126953125,
  'p95_response_time': np.float64(7.692253589630127),
  'p99_response_time': np.float64(8.469774723052979),
  'min_response_time': 2.2001266479492188,
  'max_response_time': 

In [11]:
quick_test("5-cache-aside", "light")  # 例: 単一サービスの軽負荷テストを実行

2025-09-11 20:54:25,848 - INFO - 🚀 包括的性能テスト開始 - 20250911_205425
2025-09-11 20:54:25,848 - INFO - 対象サービス: ['5-cache-aside']
2025-09-11 20:54:25,849 - INFO - テストパターン: ['light']
2025-09-11 20:54:25,850 - INFO - 
📊 5-cache-aside のテスト開始
2025-09-11 20:54:42,545 - INFO - 
進行状況: 1/1


🧪 5-cache-aside - 軽負荷 開始
   クライアント数: 10, 総リクエスト数: 1000


2025-09-11 20:54:44,988 - INFO - 💾 結果保存完了: /Users/codefox/workspace/practice_infra_arch/key_value_store/analysis/data/intermediate_5-cache-aside_light_20250911_205425.json


✅ 5-cache-aside - 軽負荷 完了
   実行時間: 2.44秒, QPS: 410.63


2025-09-11 20:55:15,500 - INFO - 💾 結果保存完了: /Users/codefox/workspace/practice_infra_arch/key_value_store/analysis/data/comprehensive_test_20250911_205425.json
2025-09-11 20:55:15,501 - INFO - 
🎉 全テスト完了! 結果は /Users/codefox/workspace/practice_infra_arch/key_value_store/analysis/data に保存されました


[{'total_requests': 1000,
  'successful_requests': 1000,
  'error_rate': 0.0,
  'qps': 410.6258428037701,
  'avg_response_time': 2.4127771854400635,
  'median_response_time': 1.7805099487304688,
  'p95_response_time': np.float64(5.479323863983154),
  'p99_response_time': np.float64(6.927461624145508),
  'min_response_time': 1.2488365173339844,
  'max_response_time': 12.588024139404297,
  'initial_stats': {},
  'final_stats': {},
  'service_name': '5-cache-aside',
  'pattern_name': 'light',
  'pattern_description': '軽負荷',
  'timestamp': '2025-09-11T20:54:44.986978',
  'clients': 10,
  'duration': 2.435307025909424}]

## 📊 結果について

テスト結果は以下の場所に保存されます：

- **JSONファイル**: `analysis/data/*.json` - 完全な結果データ
- **CSVファイル**: `analysis/data/*.csv` - 数値データのみ（分析用）

結果の分析には `analyze_results.ipynb` ノートブックを使用してください。

### 測定項目
- **レスポンス時間**: 平均、中央値、95%tile、99%tile
- **スループット**: QPS（Query Per Second）
- **エラー率**: 失敗したリクエストの割合
- **リソース使用量**: CPU、メモリ使用量（可能な場合）

### 注意事項
- テスト実行中は他のアプリケーションの使用を控えてください
- 高負荷テストは時間がかかる場合があります
- Docker環境が正常に動作することを事前に確認してください